In [ ]:
mesyac = '1'
date_parser = '2025/' + mesyac
test_date = 'https://telecombloger.ru/date/'+date_parser
test_date

In [ ]:
import asyncio
from pyppeteer import launch
import os
import pandas as pd
import re

async def fetch_articles():
    browser = await launch(
        headless=True,
        executablePath=r"C:\Program Files\Google\Chrome\Application\chrome.exe"
    )
    page = await browser.newPage()
    await page.goto(test_date, {'waitUntil': 'networkidle2'}) # Здесь задаем сайт
    await page.setViewport({'width': 1280, 'height': 800})
    await asyncio.sleep(5)  # задержка перед началом парсинга

    # Извлечение всех блоков article и нужных данных
    articles = await page.evaluate('''() => {
        return Array.from(document.querySelectorAll('article')).map(article => {
            const idMatch = article.id.match(/post-(\\d+)/);
            const post = idMatch ? idMatch[1] : null;

            const aTag = article.querySelector('a.news-item-title-link');
            const href = aTag ? aTag.href : null;

            const catTags = Array.from(article.querySelectorAll('.news-item-categories a'))
                                 .map(el => el.textContent.trim());

            const dateEl = article.querySelector('.meta-published');
            const date = dateEl ? dateEl.textContent.trim() : null;

            return {
                post: post,
                href: href,
                category_tag: catTags,
                date: date
            };
        });
    }''')

    await browser.close()
    return articles

def save_to_dataframe(data):
    df = pd.DataFrame(data)
    df['category_tag'] = df['category_tag'].apply(lambda tags: ', '.join(tags))
    return df

In [ ]:
async def extract_article_text(url):
    from pyppeteer.errors import NetworkError, PageError
    try:
        browser = await launch(
            headless=True,
            executablePath=r"C:\Program Files\Google\Chrome\Application\chrome.exe",
            args=['--no-sandbox']
        )
        page = await browser.newPage()
        await page.goto(url, {'waitUntil': 'domcontentloaded'})
        
        # Явное ожидание нужного блока
        await page.waitForSelector('.news-page-text', {'timeout': 10000})
        await asyncio.sleep(2)  # небольшая задержка

        text = await page.evaluate('''() => {
            const content = document.querySelector('.news-page-text');
            return content ? content.innerText.trim() : null;
        }''')

        await browser.close()
        return text

    except NetworkError as e:
        print(f"[!] Сетевая ошибка: {e}")
    except PageError as e:
        print(f"[!] Ошибка страницы: {e}")
    except Exception as e:
        print(f"[!] Неизвестная ошибка: {e}")
    finally:
        try:
            await browser.close()
        except:
            pass
    return None


In [ ]:
async def main():
    print("[*] Парсинг сайта telecombloger.ru...")
    articles = await fetch_articles()
    df = save_to_dataframe(articles)
    print("[✓] Готово. DataFrame создан.")
    return df

df = await main()
df.head() # Создаем DataFrame


In [ ]:
df

In [ ]:
x = df.index.max()  # максимальный индекс
for i in range(x, -1, -1):  # от x до 0 включительно
    test_url = "https://telecombloger.ru/" + df.iloc[i].post
    print(f"[*] Извлекаем текст - {test_url}")

    url = test_url
    text = await extract_article_text(url)

    if text:
        # print("[✓] Извлечённый текст:\n")
        # print(text[:1000] + '...')

        # Имя файла на основе ссылки
        filename = df.iloc[i].post.replace("/", "_") + ".txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(text)
        print(f'[✓] Файл сохранен: {filename}')
    else:
        print(f"[!] Не удалось извлечь текст для: {url}")


In [ ]:
# Сохранение в CSV
df.to_csv('telecombloger_2025_' + mesyac + '.csv', index=False, encoding='utf-8-sig')